                                SEGEMTACION DE MANZANAS MEDIANTE UMBRALIZACION
La segmentacion es uno de los procesos primordiales en el preprocesamiento de imagenes a la hora de sacar informacion significativa contenida en un imagen.

En este notebook se implemento una segmentacion de las mas basica que es por medio de la umbralizacion de la imagen a si como utilizacion de operaciones morfologicas para que el umbralizado este lo mas limpio posible de riudo en la imagen.

La base de datos que se utilizo se compone de la siguiente manera esta compuesto por 3 carpetas en las cuales en cada uno tiene 8 subcarpetas, donde estan alamcenadas 4 distintos tipos distintos de manzanas y cada tipo con su respetivo manzana sana y descompueta. El total de la imagenes es de 4,800 imaganes de tamaño 800 x 600 pixeles 

In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pandas as pd
import csv
#Estas son todas la blibliotecas de python a utilizar en esta implementacion 

Datos=[]  #Esta variable es la encargarda de hacer la tarea de funcionar como un framedata para poder almacenarlo despues dentro de un CSV
Encabezado="Nombre de la Imagen,X,Y,W,H\n" # Con esta lista se pone los encabezados de la lista de datos
Datos.append(Encabezado)



direcciones= 'C:~\\IMAGENES_MANZANAS_SD' # este es el path donde esta ubicado las imagenes de entrada que se desean segmentar
ListaCarpetas=os.listdir(direcciones) # mediante el comando os.listdir se obtiene la lista direcciones de todo el contenido 
                                      #que exite en ese diretorio 

for fichero in ListaCarpetas: # Con esto se accede a cada uno de las carpetas de 
    List1=os.listdir(os.path.join(direcciones,fichero)) # se vuelve a acceder al contenido de cada carpeta
    for fi2 in List1:
        List2=os.listdir(os.path.join(direcciones,fichero,fi2))
        for fi3 in List2:  # se accede a cada imagen de esas subcarpetas 
            img_color=cv2.imread(os.path.join(direcciones, fichero,fi2, fi3)) # se lee a cada uno de las imagenes contenida en cada subcarpeta
            b,g,r = cv2.split(img_color) # se separan los 3 canales de la imagen mediante la funcion split
            img_filtrada=cv2.GaussianBlur(b,(7,7),5) # se filtra el canal azul mediante un filtro Gaussiano con kernel de 7X7
            ret3, inverse_mask = cv2.threshold(img_filtrada,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) # Se umbraliza la imagen usando Otsu para una umbralizacion automatica
            mask = cv2.bitwise_not(inverse_mask) # se invierte los valores de la imagen 
            kernel = np.ones((7,7),np.uint8) # Creacion de un Kernel de 7x7 
            mask_morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # se hace un operacion morfologica cerrada para darle uniformidad a la mascara
            kernel2 = np.ones((7,7),np.uint8) #Creaccion de un Kernel de 7x7
            mask_morph2 = cv2.morphologyEx(mask_morph, cv2.MORPH_OPEN, kernel2) # se hace una operacion mofonogica open para parse uniformidad a la region 
            char='mask_'+fi3+'.png' # variable para nombrar la imagen 
            actual=os.getcwd() # se busca la direccion actual de trabajo para poder moverse a otra la carpeta donde esta la imagen y poder guardar la imagen de resultado en la misma ubicacion
            os.chdir(os.path.join(actual,'IMAGENES_MANZANAS_SD',fichero,fi2)) # con este se entra a la carpeta de la imagen
            cv2.imwrite(char,mask_morph2) # se guarda la imagen 
            contorno,jerarquia=cv2.findContours(mask_morph,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # se buca los contornos de la imagen umbralizada mediante la funcion findContours con los siguiente parametros 
            areas = [cv2.contourArea(c) for c in contorno] # se saca el area de cada contorno encontrasa en la maskara 
            max_index = np.argmax(areas) # Se busca el area mas grande que seria la region objetivo que para esta tarea es la manzana 
            cnt=contorno[max_index] # se accede a los valores de ese contorno
            x,y,w,h=cv2.boundingRect(cnt) # mediante la funcion boundingRect se calcula el centro y ancho y largo del rectangulo que rodea ese contorno
            crop_img = img_color[y:y+h, x:x+w,:] # se recorta la imagen 
            char2='Recorte'+fi3+'.png'
            cv2.imwrite(char2,crop_img)
            os.chdir(actual)
            DatoOb="\\"+str(fichero)+"\\"+str(fi2)+"\\"+str(fi3)+","+str(x)+","+str(y)+","+str(w)+","+str(h)+"\n" # se guarda en una string los valores calculados 
            Datos.append(DatoOb)# se asigan este string al framedata
file=open("ManzaNaData.csv","w") # se abre el archivo de almecenamiento de datos mediante open() 
file.writelines(Datos) # mediante la funcion writelines() se escribe todos los datos almacenados dentro de la FrameDate Datos
file.close() # se cierra el documento mediante la funcion close()